In [5]:
import pandas as pd

In [30]:
race_data = pd.read_pickle("data/race_data.pkl")

In [31]:
race_data = pd.concat([race_data, race_data["選手名府県/年齢/期別"].str.split("/", expand=True)], axis=1).drop("選手名府県/年齢/期別", axis=1)
race_data = race_data.drop(0, axis=1)

In [32]:
prediction = pd.get_dummies(race_data["予想"])
isFine = pd.get_dummies(race_data["好気合"], drop_first=True)
leg_type = pd.get_dummies(race_data["脚質"])
group = pd.get_dummies(race_data["級班"])
race_data = pd.concat([race_data, prediction, isFine, leg_type, group], axis=1).drop(["予想", "好気合", "脚質", "級班"], axis=1)
print(race_data.shape)

(57768, 30)


In [33]:
race_data = race_data.replace(["失", "落", "故", "欠"], 9)
race_data["ギヤ倍数"] = race_data["ギヤ倍数"].map(lambda x: x[:4] if len(x)>4 else x)
race_data[2] = race_data[2].map(lambda x: x.replace(" （欠車）", "") if "欠車"in x else x)
race_data["着順"] = race_data["着順"].map(lambda x: 1 if x in ["1", "2", "3"] else 0)
race_data = race_data.astype("float64")
print(race_data.shape)

(57768, 30)


In [36]:
race_y = race_data['着順']
race_x = race_data.drop('着順', axis=1)
race_x = race_x.loc[:, ['車番', '総評', '枠番', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外', 1, 2, 'nan', '×', '▲', '△', '○', '◎', '注', '★', '両', '追', '逃', 'A1', 'A2', 'A3', 'L1', 'S1', 'S2', 'SS']]

,車番,総評,枠番,ギヤ倍数,競走得点,1着,2着,3着,着外,1,...,両,追,逃,A1,A2,A3,L1,S1,S2,SS
2720210401010001,3.0,1.0,3.0,3.92,78.33,9.0,8.0,6.0,3.0,37.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2720210401010001,2.0,3.0,2.0,3.86,72.77,4.0,5.0,4.0,14.0,49.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2720210401010001,1.0,3.0,1.0,3.92,73.66,16.0,15.0,7.0,30.0,37.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2720210401010001,5.0,3.0,5.0,3.92,70.50,3.0,6.0,10.0,29.0,32.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2720210401010001,4.0,5.0,4.0,3.85,67.45,1.0,5.0,10.0,26.0,46.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8320210728020012,6.0,5.0,6.0,3.92,98.90,3.0,12.0,12.0,36.0,44.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8320210728020012,2.0,2.0,2.0,3.92,110.62,14.0,3.0,5.0,33.0,42.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8320210728020012,4.0,4.0,4.0,3.92,101.88,15.0,10.0,7.0,39.0,39.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8320210728020012,3.0,4.0,3.0,3.93,100.15,3.0,8.0,8.0,33.0,44.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
race_x.to_pickle("data/race_x.pkl")
race_y.to_pickle("data/race_y.pkl")